# Intention

## Extracting transitive verbs (transitive verbs - verbs that require objects) and direct objects.

In [1]:
import spacy

nlp = spacy.load('en_core_web_lg')

doc = nlp(u'show me the best hotel in berlin')

for token in doc:
    if token.dep_ == 'dobj':
        print(token.head.text + token.text.capitalize())

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)
    if ent.label_ == "GPE":
        print(ent.text)

showHotel
berlin 26 32 GPE
berlin


## Sentence Sequences - Conjunctions

In [2]:
#apply the pipeline to the sample sentence
doc = nlp(u'I want to place an order for a pizza.')

# extract the direct object and its transitive verb
dobj = ''
tverb = ''
for token in doc:
    if token.dep_ == 'dobj':
        dobj = token
        tverb = token.head

# extract the verb for the intent's definition
intentVerb = ''
verbList = ['want', 'like', 'need', 'order']
if tverb.text in verbList:
    intentVerb = tverb
else:
    if tverb.head.dep_ == 'ROOT':
        intentVerb = tverb.head

# extract the object for the intent's definition
intentObj = ''
objList = ['pizza', 'cola']
if dobj.text in objList:
    intentObj = dobj
else:
    for child in dobj.children:
        if child.dep_ == 'prep':
            intentObj = list(child.children)[0]
            break
        elif child.dep_ == 'compound':
            intentObj = child
            break

# print the intent expressed in the sample sentence
print(intentVerb.text + intentObj.text.capitalize())

wantPizza


## Recognizing Synonyms

In [3]:
doc = nlp(u'I want a dish.')

#extract the transitive verb and its direct object from the dependency tree
for token in doc:
    if token.dep_ == 'dobj':
        verb = token.head.text
        dobj = token.text

#create a list of tuples for possible verb synonyms
verbList = [('order','want','give','make'),('show','find')]

#find the tuple containing the transitive verb extracted from the sample
verbSyns = [item for item in verbList if verb in item]

#create a list of tuples for possible direct object synonyms
dobjList = [('pizza','pie','dish'),('cola','soda')]

#find the tuple containing the direct object extracted from the sample
dobjSyns = [item for item in dobjList if dobj in item]

#replace the transitive verb and the direct object with synonyms supported by the application
#and compose the string that represents the intent
intent = verbSyns[0][0] + dobjSyns[0][0].capitalize()
print(intent)

orderPizza


## Semantic Similarity

In [4]:
doc = nlp(u'I feel like eating a pie')
doc2 = nlp(u'food')

for token in doc:
    if token.dep_ == 'dobj':
        dobj = token

if dobj.similarity(doc2[0]) > 0.4:
    print("Would you like to look at our menu?")

print(dobj.similarity(doc2[0]))

Would you like to look at our menu?
0.4594937562942505


## Recognizing intention from sentence sequences

In [5]:
doc = nlp(u'I have finished my pizza. I want another one.')

verbList = [('order','want','give','make'),('show','find')]
dobjList = [('pizza','pie','pizzaz'),('cola','soda')]
substitutes = ('one','it','same','more')
intent = {'verb': '', 'dobj': ''}

for sent in doc.sents:
    for token in sent:
        if token.dep_ == 'dobj':
            verbSyns = [item for item in verbList if token.head.text in item]
            dobjSyns = [item for item in dobjList if token.text in item]
            substitute = [item for item in substitutes if token.text in item]
            if (dobjSyns != [] or substitute != []) and verbSyns != []:
                intent['verb'] = verbSyns[0][0]
            if dobjSyns != []:
                intent['dobj'] = dobjSyns[0][0]

intentStr = intent['verb'] + intent['dobj'].capitalize()
print(intentStr)

orderPizza
